In [ ]:
import os
import ast
import numpy as np

In [ ]:
### open Hamiltonian data ###

working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir) # gets directory where running python file is!

data_dir = os.path.join(parent_dir, 'Molecular_Hamiltonian_data')
hamiltonian_data = os.path.join(data_dir, 'hamiltonians.txt')

In [ ]:
with open(hamiltonian_data, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

In [ ]:
# molecule_key = 'H1-Li1-O1_STO-3G_singlet'
molecule_key = 'H2_6-31G_singlet'
# molecule_key = 'H1-He1_3-21G_singlet_1+'
transformation, N_qubits, Hamilt_dictionary, _ ,_, _ = hamiltonians[molecule_key]
del hamiltonians

# 1. Get OpenFermion representation of Hamiltonian

In [ ]:
from quchem.Misc_functions.conversion_scripts import Get_Openfermion_Hamiltonian

openFermion_H = Get_Openfermion_Hamiltonian(Hamilt_dictionary)
openFermion_H

# 2. Get cliques defined by commutativity 


In [ ]:
from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
Graph_colouring_strategy='largest_first'


anti_commuting_sets = Clique_cover_Hamiltonian(openFermion_H, 
                                                     N_qubits, 
                                                     commutativity_flag, 
                                                     Graph_colouring_strategy)
anti_commuting_sets

# 3. Example of X_sk operator

### lexicographica order
(maximises circuit reductions)

In [ ]:
key_larg, largest_AC_set = max(anti_commuting_sets.items(), key=lambda x:len(x[1])) 

In [ ]:
largest_AC_set

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Build_R_SeqRot_Q_circuit_manual_Reduced

full_RS_circuit, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_manual_Reduced(largest_AC_set,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Build_R_SeqRot_Q_circuit_IBM_Reduced

IBM_full_RS_circuit, IBM_Ps, IBM_gamma_l = Auto_Build_R_SeqRot_Q_circuit_IBM_Reduced(largest_AC_set,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
IBM_full_RS_circuit

In [ ]:
from quchem.Misc_functions.Misc_functions import count_circuit_gates
print(count_circuit_gates(full_RS_circuit))
print(count_circuit_gates(IBM_full_RS_circuit))

In [ ]:
from quchem.Misc_functions.Misc_functions import Get_circuit_depth
print('quantum circuit depth:', Get_circuit_depth(full_RS_circuit))
print('IBM quantum circuit depth:', Get_circuit_depth(IBM_full_RS_circuit))

In [ ]:
np.array([i**2 for i in range(10)])% 1

# linear algebra expermient

In [ ]:
from openfermion.linalg import qubit_operator_sparse
from scipy.linalg import eigh

H_matrix = qubit_operator_sparse(openFermion_H)
eig_values, eig_vectors = eigh(H_matrix.todense()) # NOT sparse!

idx = eig_values.argsort()  
eigenValues = eig_values[idx]
eigenVectors = eig_vectors[:,idx]

ground_state = np.around(eigenVectors[:,0].real, 10)

In [ ]:
ground_state.conj().T @ H_matrix.todense() @ ground_state

In [ ]:
min(eigenValues)

In [ ]:
from quchem.Qcircuit.Circuit_functions_to_create_arb_state import intialization_circuit
ansatz_circuit, global_phase = intialization_circuit(ground_state,
                             0,
                             check_circuit=False)

print('global_phase =', global_phase)
ansatz_circuit

In [ ]:
np.allclose(ground_state, ansatz_circuit.final_state_vector()*global_phase)

In [ ]:
from quchem.Qcircuit.Circuit_functions_to_create_arb_state import prepare_arb_state_cirq_matrix_gate
ansatz_circuit = prepare_arb_state_cirq_matrix_gate(ground_state)
ansatz_circuit

In [ ]:
np.allclose(ground_state, ansatz_circuit.final_state_vector())

In [ ]:
final_state = ansatz_circuit.final_state_vector()
np.trace(np.outer(final_state, final_state)@H_matrix)
# ansatz_circuit.final_state_vector().conj().T @ H_matrix.todense() @ ansatz_circuit.final_state_vector()

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Seq_Rot_VQE_Experiment_UP_manual_reduced_circuit_lin_alg

manual_reduction_lin_alg_SeqRot_exp = Auto_Seq_Rot_VQE_Experiment_UP_manual_reduced_circuit_lin_alg(
anti_commuting_sets,
ansatz_circuit)

E_SeqRot_manual_circuit_reduction = manual_reduction_lin_alg_SeqRot_exp.Calc_Energy(check_circuit=True,
                                                                                    check_reduction_lin_alg=True, 
                                                                                    maximise_CNOT_reduction=True)
E_SeqRot_manual_circuit_reduction

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Seq_Rot_VQE_Experiment_UP_IBM_reduced_circuit_lin_alg

IBM_reduction_lin_alg_SeqRot_exp = Auto_Seq_Rot_VQE_Experiment_UP_IBM_reduced_circuit_lin_alg(
anti_commuting_sets,
ansatz_circuit,
allowed_qiskit_gates=['id', 'rz', 'ry', 'rx', 'cx' ,'s', 'h', 'y','z'],
IBM_opt_lvl=3)

E_SeqRot_IBM_circuit_reduction = IBM_reduction_lin_alg_SeqRot_exp.Calc_Energy(check_circuit=True)
E_SeqRot_IBM_circuit_reduction

In [ ]:
from openfermion import QubitOperator
Ps = QubitOperator('Y1 X2', 0.2)
PauliStr_Ps, beta_S = tuple(*Ps.terms.items())
PauliStr_Ps_Z = [(qNo, 'Z')for qNo, Pstr in PauliStr_Ps]
Ps_Zchange = QubitOperator(PauliStr_Ps_Z, beta_S)
Ps_Zchange

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions import Seq_Rot_VQE_Experiment_UP_circuit_lin_alg

exp_linalg_pure = Seq_Rot_VQE_Experiment_UP_circuit_lin_alg(anti_commuting_sets, ansatz_circuit, S_key_dict=None)
exp_linalg_pure.Calc_Energy()

In [ ]:
min(eigenValues)

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced

full_RS_circuit_tiket, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced(anti_commuting_sets[1],
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit_tiket

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Full_SeqRot_auto_Rl_Circuit_tiket_Reduced

full_circuit_tiket, Ps, gamma_l = Full_SeqRot_auto_Rl_Circuit_tiket_Reduced(
                                                      ansatz_circuit,
                                                      anti_commuting_sets[1],
                                                      N_qubits, 
#                                                       check_reduction_lin_alg=True, 
#                                                       check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_circuit_tiket

In [ ]:
full_RS_circuit_tiket, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced(largest_AC_set,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit_tiket

In [ ]:
from quchem.Misc_functions.Misc_functions import count_circuit_gates
print(count_circuit_gates(full_RS_circuit))
print(count_circuit_gates(IBM_full_RS_circuit))
print()
print(count_circuit_gates(full_RS_circuit_tiket))

In [ ]:
from quchem.Unitary_Partitioning.Seq_Rot_circuit_functions_AUTO import Auto_Seq_Rot_VQE_Experiment_UP_tiket_reduced_circuit_lin_alg

exp = Auto_Seq_Rot_VQE_Experiment_UP_tiket_reduced_circuit_lin_alg(anti_commuting_sets, ansatz_circuit)
exp.Calc_Energy(check_reduction_lin_alg=True, check_circuit=True)

In [ ]:
min(eigenValues)

In [ ]:
from copy import deepcopy
def lexicographical_sort_BASIS_MATCH(list_P_ops):
    """
    maximises adjacent single qubit pauli terms in Pauliword list (allowing best change of basis cancellation)
    """
    fullOp = reduce(lambda Op1, Op2: Op1+Op2, list_P_ops)
    max_qubits = count_qubits(fullOp)

    P_Words = []
    for op in list_P_ops:
        
#         Q_Nos, P_strings = zip(*list(*op.terms.keys()))
#         P_dict = dict(zip(Q_Nos, P_strings)) # zip(keys, values)

        P_dict =  dict(tuple(*op.terms.keys())) 
        arr = [P_dict.get(qNo, 'I') for qNo in range(max_qubits)]
          
        P_Words.append(arr)
    
    P_Words_copy = deepcopy(P_Words)
    re_ordered_ind =[]
    sorted_list = []
    while P_Words!=[]:
        if sorted_list==[]:
            ind_match=0
        else:
            op_prev = sorted_list[-1] # take last sorted term
            
            # get similarity in binary and sum array
            # the larger the int the better the match between sigma terms!
            similarity_list = [(op_j,sum((np.array(op_prev)==np.array(op_j)).astype(int))) for op_j in P_Words if op_j != op_i]
            largest_match = max(similarity_list, key=lambda x:x[1])
            ind_similarity_list = similarity_list.index(largest_match)

            op_j = similarity_list[ind_similarity_list][0]
            ind_match = P_Words.index(op_j)
            
        op_i = P_Words.pop(ind_match)
        sorted_list.append(op_i)
        re_ordered_ind.append(P_Words_copy.index(op_i))


    lex_sorted = (np.array(list_P_ops)[re_ordered_ind]).tolist()
    return lex_sorted

In [ ]:
from quchem.Unitary_Partitioning.Graph import Vector_QubitHamiltonian
from functools import reduce
from openfermion.utils import count_qubits

list_P_ops = largest_AC_set

fullOp = reduce(lambda Op1, Op2: Op1+Op2, list_P_ops)
max_qubits = count_qubits(fullOp)

Hamilt_vector = Vector_QubitHamiltonian(fullOp, max_qubits)
binary_matrix =  Hamilt_vector.binary_mat.toarray()

print(binary_matrix)
list(fullOp)

In [ ]:
# new_order = np.zeros_like(binary_matrix)
# new_order[0,:] = binary_matrix[0,:]
# for ind in range(1,binary_matrix.shape[0]):
#     Hamilt_vector.binary_mat[:,ind]

new_order = binary_matrix[0].copy().reshape(1, binary_matrix.shape[1])
binary_matrix =np.delete(binary_matrix, 0, 0)
# while new_order.shape[0]!=len(list_P_ops):
while binary_matrix.shape[0]>0:
    score_list=[]
    for P_vec in binary_matrix:
        similarity=0
        for rev_ind in list(range(new_order.shape[0]-1, -1, -1)):
            P_last = new_order[rev_ind]
            similarity += sum((P_vec==P_last).astype(int))
            
            I_locations = np.intersect1d(np.where(P_last[:max_qubits]==0)[0], np.where(P_last[max_qubits:]==0)[0])
            ind_sympletic = np.hstack((I_locations, max_qubits+I_locations))
            
            empty = np.zeros_like(P_vec)
            empty[ind_sympletic] = P_vec[ind_sympletic]
            P_vec = empty
            
        score_list.append(similarity)
    
    print(score_list)
    largest_ind = score_list.index(max(score_list))
    new_order = np.vstack((new_order, binary_matrix[largest_ind]))
    binary_matrix =np.delete(binary_matrix, largest_ind, 0)
    

In [ ]:
(np.array([1,0,0,0]) + np.array([1,0,0,0]))/2

In [ ]:
for P_symp in new_order:
    P_str =''
    for i in range(max_qubits):
        X_term=P_symp[i]
        Z_term=P_symp[i+max_qubits]
        
        if (X_term+Z_term)==1:
            P_str += 'Y'
        elif X_term == 1:
            P_str += 'X'
        elif Z_term == 1:
            P_str += 'Z'
        else:
            P_str += 'I'
    print(P_str)
    

In [ ]:
for P_symp in Hamilt_vector.binary_mat.toarray():
    P_str =''
    for i in range(max_qubits):
        X_term=P_symp[i]
        Z_term=P_symp[i+max_qubits]
        
        if (X_term+Z_term)==1:
            P_str += 'Y'
        elif X_term == 1:
            P_str += 'X'
        elif Z_term == 1:
            P_str += 'Z'
        else:
            P_str += 'I'
    print(P_str)
    

In [ ]:
lexicographical_sort_BASIS_MATCH(largest_AC_set)

In [ ]:
I_locations = np.where(P_last[:max_qubits]==P_last[max_qubits:])[0]
P_vec[] 

In [ ]:
P_last[np.hstack((I_locations, 2*I_locations))]

In [ ]:
P_vec[]

In [ ]:
new_order

In [ ]:
np.vstack((np.array([1,2,3]), np.array([1,2,3])))

In [ ]:
(new_order[0,:]==new_order[1,:]).astype(int)

In [ ]:
new_order = binary_matrix[0:2,:]
new_order

In [ ]:
A = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])

A = np.delete(A, 2, 0)
A

In [ ]:
for row in A:
    print(row)

In [ ]:
binary_matrix[0].shape

In [ ]:
Hamilt_vector.binary_mat[0,:]

In [ ]:
running_list = ['XXXI', 'ZIZZ']

outer_most_term = 

[ind for ind, sig in enumerate('IIZZ') if sig=='I']

In [ ]:
largest_AC_set

In [ ]:
const = largest_AC_set[1].terms.values()
qNos, Pstrs = zip(*list(largest_AC_set[1].terms.keys())[0])

In [ ]:
const2 = largest_AC_set[2].terms.values()
qNos2, Pstrs2 = zip(*list(largest_AC_set[2].terms.keys())[0])

In [ ]:
qNos

In [ ]:
def Count_canellations_from_list(P_list, P_active, n_qubits):
    """
    Function gives a score for how many change of basis Pauli operators can obtained for P_active
    iterating backwards through P_list. Idea is to use this to re_order terms in SeqRot to maximize
    change of basis cancellations
    
    """
    
    
    P_dict =  dict(tuple(*P_active.terms.keys())) 
    qNos_active = np.array(list(P_dict.keys()))
    Pstrs_active = np.array([P_dict.get(qNo, 'I') for qNo in range(n_qubits)])
    
    largest_canellation = len(qNos_active)
    N_cancellations=0
    for P_op in P_list[::-1]: # reverse order
        P_comp_dict =  dict(tuple(*P_op.terms.keys())) 
        Pstrs_comp = np.array([P_comp_dict.get(qNo, 'I') for qNo in range(n_qubits)])
        
        ind_to_delete=[]
        for j, act_ind in enumerate(qNos_active):
            if Pstrs_comp[act_ind]== Pstrs_active[act_ind]:
                ind_to_delete.append(j)
                N_cancellations+=1
                
            elif Pstrs_comp[act_ind] == 'I':
                continue
            else:
                # case when mismatch!
                ind_to_delete.append(j)
                # delete term BUT don't add to cancellation counter
        
        qNos_active = np.delete(qNos_active, ind_to_delete, 0)
        
    score = N_cancellations/largest_canellation
    return score

In [ ]:
from openfermion import QubitOperator
P_list = [QubitOperator('X1 X2', 1), QubitOperator('Z0', 1), QubitOperator('Z1 X2 Z4', 1)]

P_active = QubitOperator('Z0 Z2', 1)

Count_canellations_from_list(P_list, P_active, 5)

In [ ]:
from copy import deepcopy
def lexicographical_sort_BASIS_MATCH(list_P_ops, n_qubits):
    """
    maximises adjacent single qubit pauli terms in Pauliword list (allowing best change of basis cancellation)
    """
    list_P_ops = deepcopy(list_P_ops)
    final_list_size = len(list_P_ops)
    
    size_of_terms = [(ind, len(list(*op.terms.keys()))) for ind, op in enumerate(list_P_ops)]
    selected_ind = min(size_of_terms, key=lambda x:x[1])[0] # get ind of longest term
    re_ordered_P_op_list = [list_P_ops.pop(selected_ind)] # first term in longest
    
#     re_ordered_P_op_list = [list_P_ops.pop(0)] # first term is first thing in list
    
    while len(re_ordered_P_op_list)<final_list_size:

        score_list = [(ind, Count_canellations_from_list(re_ordered_P_op_list, op, n_qubits)) 
                      for ind, op in enumerate(list_P_ops)]
        

        selected_ind = max(score_list, key=lambda x:x[1])[0]
        re_ordered_P_op_list.append(list_P_ops.pop(selected_ind))

    return re_ordered_P_op_list

In [ ]:
re_ordered = lexicographical_sort_BASIS_MATCH(largest_AC_set, 5)
re_ordered

In [ ]:
def Count_basis_cancellation(P_list, P_active, n_qubits):
    """
    Function gives a score for how many change of basis Pauli operators can obtained for P_active
    iterating backwards through P_list. Idea is to use this to re_order terms in SeqRot to maximize
    change of basis cancellations
    
    """
    
    
    P_dict =  dict(tuple(*P_active.terms.keys())) 
    qNos_active = np.array(list(P_dict.keys()))
    Pstrs_active = np.array([P_dict.get(qNo, 'I') for qNo in range(n_qubits)])
    
    largest_canellation = len(qNos_active)
    N_cancellations=0
    for P_op in P_list:
        P_comp_dict =  dict(tuple(*P_op.terms.keys())) 
        Pstrs_comp = np.array([P_comp_dict.get(qNo, 'I') for qNo in range(n_qubits)])
        
        ind_to_delete=[]
        for j, act_ind in enumerate(qNos_active):
            if Pstrs_comp[act_ind]== Pstrs_active[act_ind]:
                ind_to_delete.append(j)
                N_cancellations+=1
                
            elif Pstrs_comp[act_ind] == 'I':
                continue
            else:
                # case when mismatch!
                ind_to_delete.append(j)
                # delete term BUT don't add to cancellation counter
        
        qNos_active = np.delete(qNos_active, ind_to_delete, 0)
        
    return N_cancellations

In [ ]:
N_canc=0
for ind, op in enumerate(re_ordered[:-1]):
    N_canc += Count_basis_cancellation(re_ordered[(ind+1):], op, 5)
N_canc

In [ ]:
re_ordered

In [ ]:
def Count_canellations_from_list(P_list, P_active, n_qubits):
    """
    Function gives a score for how many change of basis Pauli operators can obtained for P_active
    iterating backwards through P_list. Idea is to use this to re_order terms in SeqRot to maximize
    change of basis cancellations
    
    """
    
    
    P_dict =  dict(tuple(*P_active.terms.keys())) 
    qNos_active = np.array(list(P_dict.keys()))
    Pstrs_active = np.array([P_dict.get(qNo, 'I') for qNo in range(n_qubits)])
    
    largest_canellation = len(qNos_active)
    N_cancellations=0
    for P_op in P_list[::-1]: # reverse order
        P_comp_dict =  dict(tuple(*P_op.terms.keys())) 
        Pstrs_comp = np.array([P_comp_dict.get(qNo, 'I') for qNo in range(n_qubits)])
        
        ind_to_delete=[]
        for j, act_ind in enumerate(qNos_active):
            if Pstrs_comp[act_ind]== Pstrs_active[act_ind]:
                ind_to_delete.append(j)
                N_cancellations+=1
                
            elif Pstrs_comp[act_ind] == 'I':
                continue
            else:
                # case when mismatch!
                ind_to_delete.append(j)
                # delete term BUT don't add to cancellation counter
        
        qNos_active = np.delete(qNos_active, ind_to_delete, 0)
        
    score = N_cancellations/largest_canellation
    return score

In [ ]:
 max(out, key=lambda x:x[1])

In [ ]:
test = np.intersect1d(qNos, qNos2)
test2 = np.delete(test, 1, 0)
test2

In [ ]:
np.t

In [ ]:
P_dict =  dict(tuple(*largest_AC_set[1].terms.keys())) 
max_qubits=5
arr = [P_dict.get(qNo, 'I') for qNo in range(max_qubits)]
arr

In [ ]:
largest_AC_set_copy = deepcopy(largest_AC_set)

In [ ]:
running_order = [largest_AC_set_copy.pop(0)]

while len(running_order)< len(largest_AC_set):
    for term in largest_AC_set_copy:
        

In [ ]:
from quchem.Unitary_Partitioning.Graph import Vector_QubitHamiltonian
from functools import reduce
from openfermion.utils import count_qubits

list_P_ops = largest_AC_set

fullOp = reduce(lambda Op1, Op2: Op1+Op2, list_P_ops)
max_qubits = count_qubits(fullOp)

Hamilt_vector = Vector_QubitHamiltonian(fullOp, max_qubits)
binary_matrix =  Hamilt_vector.binary_mat.toarray()

print(binary_matrix)
list(fullOp)

In [ ]:
new_order = binary_matrix[0].copy().reshape(1, binary_matrix.shape[1])
binary_matrix =np.delete(binary_matrix, 0, 0)

while binary_matrix.shape[0]>0:
    score_list=[]
    for P_vec in binary_matrix:
        active_X_ind = np.where(P_vec[:max_qubits]>0)[0]
        active_Z_ind = np.where(P_vec[max_qubits:]>0)[0]
        max_cancellations = len(np.where((P_vec[:max_qubits]+P_vec[max_qubits:])>0))
        n_canellations=0
        for rev_ind in list(range(new_order.shape[0]-1, -1, -1)):
            P_last = new_order[rev_ind]
            similarity += sum((P_vec==P_last).astype(int))
            
            I_locations = np.intersect1d(np.where(P_last[:max_qubits]==0)[0], np.where(P_last[max_qubits:]==0)[0])
            ind_sympletic = np.hstack((I_locations, max_qubits+I_locations))
            
            empty = np.zeros_like(P_vec)
            empty[ind_sympletic] = P_vec[ind_sympletic]
            P_vec = empty
            
        score_list.append(similarity)
    
    print(score_list)
    largest_ind = score_list.index(max(score_list))
    new_order = np.vstack((new_order, binary_matrix[largest_ind]))
    binary_matrix =np.delete(binary_matrix, largest_ind, 0)
    

In [ ]:
new_order

In [ ]:
P_active = largest_AC_set[4]
print(P_active)
Q_Nos, P_strings = zip(*list(*P_active.terms.keys()))
[(Q_Nos[ind], Q_Nos[ind+1]) for ind, i in enumerate(Q_Nos[:-1])]

In [ ]:
list(set([i for tup in xxx for i in tup]))

In [ ]:
def Count_CNOT_cancellation(P_list, P_active, n_qubits):
    """
    Function gives a score for how many change of basis Pauli operators can obtained for P_active
    iterating backwards through P_list. Idea is to use this to re_order terms in SeqRot to maximize
    change of basis cancellations
    
    """
    P_list = deepcopy(P_list)
    
#     P_dict =  dict(tuple(*P_active.terms.keys())) 
    Q_Nos, P_strings = zip(*list(*P_active.terms.keys()))
    
    open_CNOT_active = [(Q_Nos[ind], Q_Nos[ind+1]) for ind, i in enumerate(Q_Nos[:-1])]
    
    P_dict =  dict(tuple(*P_active.terms.keys())) 
    Pstrs_active = np.array([P_dict.get(qNo, 'I') for qNo in range(n_qubits)], dtype=object)
    
    
    largest_canellation = len(Q_Nos)-1
    
    if largest_canellation ==0:
        # single qubit case (no CNOT cancellations possible)
        return 0
    
    running_CNOT_terms =[]
    list_Pstrs_comp = []
    seen_qubits = {}
    for com_ind, P_op in enumerate(P_list[::-1]):
            
        
        Q_Nos_compare, P_strings_comp = zip(*list(*P_op.terms.keys()))
        Q_Nos_comp=[]
        for qNo in Q_Nos_compare:
            if qNo in seen_qubits:
                break
            else:
                Q_Nos_comp.append(qNo)
        
        open_CNOT_comp = [(Q_Nos_comp[ind], Q_Nos_comp[ind+1]) for ind, i in enumerate(Q_Nos_comp[:-1])]
        running_CNOT_terms.append(open_CNOT_comp)
        
        seen_qubits = set([*list(seen_qubits), *Q_Nos_comp])
        
        P_dict_comp =  dict(tuple(*P_op.terms.keys())) 
        list_Pstrs_comp.append(np.array([P_dict_comp.get(qNo, 'I') for qNo in range(n_qubits)], dtype=object))
    
    N_cancellations=0
    for i, CNOT_term_list in enumerate(running_CNOT_terms):
        for active_tup, comp_tup in zip(open_CNOT_active, CNOT_term_list):
            Pstrs_comp = list_Pstrs_comp[i]
            if (active_tup==comp_tup) and (Pstrs_active[np.array(active_tup)]==Pstrs_comp[np.array(comp_tup)]).all():
                # checks for same indices AND same change of basis!
                N_cancellations+=1
            else:
                break
    
    score = N_cancellations/largest_canellation
    
    return score

In [ ]:
P_list = [QubitOperator('Y1 Z3 Y4 ', 1), QubitOperator('Y0 Z1 Z3 Y4', 1), QubitOperator('Y0 Z1 Z2 Y3', 1)]

In [ ]:
Count_CNOT_cancellation(P_list[:-1], P_list[-1], 5)

In [ ]:
from copy import deepcopy
def lexicographical_sort_CNOT_cancel(list_P_ops, n_qubits):
    """
    maximises adjacent single qubit pauli terms in Pauliword list (allowing best change of basis cancellation)
    """
    list_P_ops = deepcopy(list_P_ops)
    final_list_size = len(list_P_ops)
    
    size_of_terms = [(ind, len(list(*op.terms.keys()))) for ind, op in enumerate(list_P_ops)]
    selected_ind = max(size_of_terms, key=lambda x:x[1])[0] # get ind of longest term
    re_ordered_P_op_list = [list_P_ops.pop(selected_ind)] # first term in longest
    
#     re_ordered_P_op_list = [list_P_ops.pop(0)] # first term is first thing in list
    
    while len(re_ordered_P_op_list)<final_list_size:

        score_list = [(ind, Count_CNOT_cancellation(re_ordered_P_op_list, op, n_qubits)) 
                      for ind, op in enumerate(list_P_ops)]
        
        if sum(score_val for _, score_val in score_list)==0:
            # no CNOT cancel possible
            # therefore maximize basis cancel
            print(score_list)
            score_list = [(ind, Count_canellations_from_list(re_ordered_P_op_list, op, n_qubits)) 
                      for ind, op in enumerate(list_P_ops)]
            
        selected_ind = max(score_list, key=lambda x:x[1])[0]
        re_ordered_P_op_list.append(list_P_ops.pop(selected_ind))

    return re_ordered_P_op_list

In [ ]:
re_ordered = lexicographical_sort_BASIS_MATCH(largest_AC_set, 5)
re_ordered

In [ ]:
re_ordered_CNOT_cancel = lexicographical_sort_CNOT_cancel(largest_AC_set, 5)
re_ordered_CNOT_cancel

In [ ]:
largest_AC_set

In [ ]:
full_RS_circuit_tiket, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced(re_ordered_CNOT_cancel,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit_tiket
print(count_circuit_gates(full_RS_circuit_tiket))

In [ ]:
largest_AC_set

In [ ]:
full_RS_circuit_tiket, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced(re_ordered,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit_tiket
print(count_circuit_gates(full_RS_circuit_tiket))

In [ ]:
full_RS_circuit_tiket, Ps, gamma_l = Auto_Build_R_SeqRot_Q_circuit_tiket_Reduced(largest_AC_set,
                                                      N_qubits, 
                                                      check_reduction_lin_alg=True, 
                                                      atol=1e-8, rtol=1e-05, 
                                                      check_circuit=True, 
                                                      maximise_CNOT_reduction=True)
full_RS_circuit_tiket
print(count_circuit_gates(full_RS_circuit_tiket))